# ChannelTS
A `ChannelTS` object is a container for a single channel.  The data are stored in an `xarray.DataArray` and indexed by time according to the metadata provided.  Here we will make a simple `electric` channel and look at how to interogate it.

In [1]:
%matplotlib inline
import numpy as np
from mth5.timeseries import ChannelTS
from mt_metadata.timeseries import Electric, Run, Station

SyntaxError: invalid syntax (channel_ts.py, line 740)

Here create some metadata, the keys are the `time_period.start` and the `sample_rate`. 

In [ ]:
ex_metadata = Electric()
ex_metadata.time_period.start = "2020-01-01T00:00:00"
ex_metadata.sample_rate = 8.0
ex_metadata.component = "ex"
ex_metadata.dipole_length = 100.
ex_metadata.units = "millivolts"

Create `Station` and `Run` metadata

In [ ]:
station_metadata = Station(id="mt001")
run_metadata = Run(id="001")

Create "realistic" data 

In [ ]:
n_samples = 4096
t = np.arange(n_samples)
data = np.sum([np.cos(2*np.pi*w*t + phi) for w, phi in zip(np.logspace(-3, 3, 20), np.random.rand(20))], axis=0)

In [ ]:
ex = ChannelTS(channel_type="electric",
              data=data,
              channel_metadata=ex_metadata,
              run_metadata=run_metadata,
              station_metadata=station_metadata)

In [ ]:
ex

## Get a slice of the data
Here we will provide a start time of the slice and the number of samples that we want the slice to be


In [ ]:
ex_slice = ex.get_slice("2020-01-01T00:00:00", n_samples=256)

In [ ]:
ex_slice

## Plot the data
This is a work in progress, but this can be done through the `xarray` container.

In [ ]:
ex_slice.plot()

## Convert to an `xarray`
We can convert the `ChannelTS` object to an `xarray.DataArray` which could be easier to use.

In [ ]:
ex_xarray = ex.to_xarray()

In [ ]:
ex_xarray

In [ ]:
ex_xarray.plot()

## Convert to an `Obspy.Trace` object
The `ChannelTS` object can be converted to an `obspy.Trace` object.  This can be useful when dealing with data received from a mainly seismic archive like IRIS.  This can also be useful for using some tools provided by [Obspy](https://docs.obspy.org/).  

**Note** there is a loss of information when doing this because an `obspy.Trace` is based on miniSEED data formats which has minimal metadata.  

In [ ]:
ex.station_metadata.fdsn.id = "mt001"
ex_trace = ex.to_obspy_trace()

In [ ]:
ex_trace

## Convert from an `Obspy.Trace` object
We can reverse that and convert an `obspy.Trace` into a `ChannelTS`.  Again useful when dealing with seismic dominated archives.

In [ ]:
ex_from_trace = ChannelTS()
ex_from_trace.from_obspy_trace(ex_trace)

In [ ]:
ex_from_trace

In [ ]:
ex

On comparison you can see the loss of metadata information.  

## Calibrate 

Removing the instrument response to calibrate the data is an important step in processing the data.  A convenience function `ChannelTS.remove_instrument_response` is supplied just for this.  
Currently, it will calibrate the whole time series at once and therefore may be slow for large data sets.  

**SEE ALSO**: `Make Data From IRIS` examples for working examples.

In [ ]:
help(ex.remove_instrument_response)

## Decimate and Filters

A common practice when working with time series would be decimating or downsampling the data.  `xarray` has some builtins for resampling, however these do not apply a filter prior to downsampling and has alias issues.  We have added some utilities for decimation and filtering following the package [xr-scipy](https://github.com/fujiisoup/xr-scipy/tree/master/xrscipy).  When MTH5 is initiated a `filt` accessor to `xarray.DataArray` and `xarray.Dataset` which includes some filtering methods as well as decimation.  Therefore for access to these methods use `DataArray.filt.decimate` or `Dataset.filt.decimate`.  There are methods for applying `lowpass`, `highpass`, `bandpass`, and `decimate`.  `ChannelTS` uses `ChannelTS._ts.filt.decimate(new_sample_rate)` for decimation.   

<div class="alert alert-block alert-info">
<b>Note:</b> In future versions of MTH5 filters will be added to ChannelTS.
</div>

### Decimate

Here we will decimate to a new sample rate of 1 sample per second.  

In [ ]:
decimated_ex = ex.decimate(1)

In [ ]:
decimated_ex.plot()

## Merge Channels

A common step in working with time series would be to combine different segments of collected for the same channel. If the sample rates are the same you can use `channel_01 + channel_02`.  This should also work if the sample rates are not the same, though you should use `ChannelTS.merge` if the sample rates are not the same. The channels combined must have the same component.  There are two builtin methods to combine channels those are `+` and `merge()`.  

  1. `added_channel = cnahhel_01 + channel_02`
  2. `merged_channel = channel_01.merge(channel_02)` 
  
Both methods use `xarray.combine_by_coords([ch1, rch2], combine_attrs='override'`.  The `combine_by_coords` method simply concatenates along similar dimensions and cares nothing of a monotonix dimension variable.  Therefore, `xarray.DataArray.reindex` is used to create a monotonically increasing time series.  Any gaps are filled interpolated using a 1-D interpolation.  The default method is `slinear` which is probably the most useful for processing time series.  If you want more control over the interpolation method use `ChannelTS.merge([ch1, ch2, ...], gap_method=interpolation_method`.  For more information on interpolation methods see [Scipy.interpolate.interp1d](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html). Similarly if you want more control on how the datasets are merged use [xarray tools](https://docs.xarray.dev/en/stable/user-guide/combining.html).  

### Add Channels

Adding channels together does 2 at a time so if you are adding multiple channels together `channel_01 + channel_02 + channel_03 ...` its better to use `merge`.  Also if you want more control on how the channels are merged and how time gaps are interpolated use `merge`.

In [ ]:
ex2 = ex.copy()
ex2.start = "2020-01-01T00:08:45"

In [ ]:
added_channel = ex + ex2

In [ ]:
added_channel

In [ ]:
added_channel.plot()

### Merge Channels And Resample

If channels have different sample rates or you want to combine channels and resample to a lower sample rate, use the keyword argument `new_sample_rate`.

In [ ]:
merged_channel = ex.merge(ex2, new_sample_rate=1)

In [ ]:
merged_channel

In [ ]:
merged_channel.plot()

In [ ]:
c = ex.decimate(1) + ex2.decimate(1)

In [ ]:
c.plot()

In [ ]:
a = ex2.decimate(1)

In [ ]:
ex.plot()
a.plot()

In [ ]:
ex2

In [ ]:
ex.copy()

In [ ]:
ex.channel_metadata.sample_rate